Откуда берутся датасеты? Практический проект по сбору данных и работе с текстами
Цель: В этом домашнем задании вам предстоит обойти все ловушки серверов, пробраться сквозь страницы html-код, собрать себе свой собственный датасет и натренировать на нём модель.

Часть 1. Парсинг

По аналогии с занятием, возьмите интересующий вас сайт, на котором можно пособирать какие-то данные (и при этом API не предоставляется).
Напишите свой парсер, который будет бегать по страничкам и автоматически что-то собирать.

In [1]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек 
import time          # Библиотека для времени

# подгрузим один из методов этой библиотеки
from fake_useragent import UserAgent

from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from itertools import groupby
import re
# подгрузим один из методов этой библиотеки
from fake_useragent import UserAgent

In [2]:
link = 'https://cdek.market/'

In [3]:
main_page = 'https://cdek.market/c/307/telefony-i-aksessuary/'

In [4]:
def get_soup(page_link:str):
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    if not response.ok:
        return []
    
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [5]:
electronik_soup = get_soup(main_page)

In [6]:
def get_ref(div_class, soup):
    _refs = []
    try:
        refs = soup.find("div", attrs={'class':div_class})
        refs = refs.findAll("a")
        for cat in refs:
            if 'href' in cat.attrs: 
                _refs.append(cat.attrs['href'])
        _refs = [el for el, _ in groupby(_refs)]
    except:
        print("Something went wrong")
    finally: 
        return _refs

In [7]:
categories = get_ref('ty-pagination__items', electronik_soup)

In [8]:
categories

['https://cdek.market/c/307/telefony-i-aksessuary/page-2/',
 'https://cdek.market/c/307/telefony-i-aksessuary/page-3/',
 'https://cdek.market/c/307/telefony-i-aksessuary/page-4/',
 'https://cdek.market/c/307/telefony-i-aksessuary/page-5/',
 'https://cdek.market/c/307/telefony-i-aksessuary/page-6/',
 'https://cdek.market/c/307/telefony-i-aksessuary/page-7/',
 'https://cdek.market/c/307/telefony-i-aksessuary/page-8/']

Достаем инфу по товарам

In [9]:
category_refs = []
for category in categories:
    product_soup = get_soup(category)
    _category_refs = get_ref('grid-list', product_soup)
    category_refs = category_refs + _category_refs

In [10]:
category_refs[0]

'https://cdek.market/p/231002/kreplenie-smartfona-na-armaturu-scepleniya-tormoza-x-grip-korotkiy-rychag-nu-trubu-s-u-skoboy-v-sbore/'

In [11]:
product_soup = get_soup(category_refs[15])
    

In [12]:
name = product_soup.find("h1").text
name

'Крепление смартфона на струбцине X-Grip® (короткий рычаг, в сборе)'

In [13]:
text_about = product_soup.find("div", attrs={'class':"ty-wysiwyg-content content-description"}).text.strip()
text_about[:500]

'Крепление состоит из универсального держателя с подпружиненным зажимом, который на шарнирном соединении через короткий рычаг 60мм., соединен с креплением-струбциной, которая может быть установлена на круглые/овальные трубы диаметром от 1,6 см до 3,8 см (0,625"- 1,5"). Универсальный держатель имеет искусную и умную конструкцию с четырьмя подпружиненными опорами, которые надежно фиксируют смартфон с достаточной силой, не закрывая собой экран. В нашем магазине большой ассортимент креплений Rammount'

In [14]:
amount = product_soup.find("span", attrs={"class":"ty-qty-in-stock ty-control-group__item"}).text.strip()
num_list = int(re.sub('[^0-9]+', '', amount))
num_list

17

In [15]:
def get_product_info(product_link):
    try:
        product_soup = get_soup(product_link)
        name = product_soup.find("h1").text.strip()
        text_about = product_soup.find("div", attrs={'class':"ty-wysiwyg-content content-description"}).text.strip()
        amount = product_soup.find("span", attrs={"class":"ty-qty-in-stock ty-control-group__item"}).text.strip()
        price = product_soup.find("span", attrs={"class":"ty-price-num"}).text.strip()
        amount = re.sub('[^0-9]+', '', amount)
        price = re.sub('[^0-9.]+', '', price)
    except:
        name = ''
        text_about = ''
        amount = '0'
        price = '0'
    return {'name':name, 'text':text_about, 'amount':amount, 'price':price}

In [16]:
def get_products_info_df(main_page):
    products_info = []
    electronik_soup = get_soup(main_page)
    pages = get_ref('ty-pagination__items', electronik_soup)
    pages.append(main_page)
    for page in pages:
        page_soup = get_soup(page)
        category_refs = get_ref('grid-list', page_soup)
        for category_ref in category_refs:
            _info = {}
            _info = get_product_info(category_ref)
            if _info['name'] and _info not in products_info:
                product_info = {}
                product_info['category'] = category_ref
                product_info.update(_info)
                products_info.append(product_info)
            time.sleep(0.3)
    products_df = pd.DataFrame(products_info)
    return products_df

In [26]:
products_df = get_products_info_df(main_page)

In [32]:
products_df = products_df.drop_duplicates(['name','text','price'], keep='last')

In [33]:
products_df.head(10)

,category,name,text,amount,price
3,https://cdek.market/index.php?dispatch=product...,Крепление смартфона на арматуру сцепления/торм...,Универсальное крепление состоит из держателя с...,6,6800.00
7,https://cdek.market/index.php?dispatch=product...,Крепление смартфона на арматуру руля X-Grip® (...,Крепление состоит из универсального держателя ...,6,6440.00
11,https://cdek.market/index.php?dispatch=product...,Крепление смартфона на U-скобе X-Grip® (коротк...,Универсальное крепление состоит из держателя с...,13,5810.00
15,https://cdek.market/index.php?dispatch=product...,Крепление смартфона на струбцине X-Grip® (коро...,Крепление состоит из универсального держателя ...,17,7610.00
19,https://cdek.market/index.php?dispatch=product...,Крепление смартфона на U-скобе X-Grip® (станда...,Универсальное крепление RAM-B-149Z-A-UN10U сос...,13,6210.00
23,https://cdek.market/index.php?dispatch=product...,Крепление смартфона на U-скобе X-Grip® (станда...,Универсальное крепление состоит из держателя с...,13,5830.00
27,https://cdek.market/index.php?dispatch=product...,Держатель смартфона X-Grip®,Универсальный держатель RAM-HOL-UN10BU имеет и...,32,3460.00
31,https://cdek.market/index.php?dispatch=product...,Держатель смартфона X-Grip®,Универсальный держатель RAM-HOL-UN7BU имеет ис...,22,3070.00
35,https://cdek.market/index.php?dispatch=product...,Крепление смартфона на присоске X-Grip® (станд...,Универсальное крепление RAM-B-166-UN10U состои...,25,6440.00
39,https://cdek.market/index.php?dispatch=product...,Крепление смартфона на адаптере AMP X-Grip® (с...,Универсальное крепление состоит из держателя с...,4,5380.00


In [34]:
products_df.to_csv('~/parse.csv')

Часть 2. NLP

1. Разбейте собранные данные на train/test, отложив 20-30% наблюдений для тестирования
2. Примените tf-idf преобразование для текстового описания. Используйте как отдельные токены, так и биграммы, отсейте стоп-слова, а также слова, которые встречаются слишком редко или слишком часто (параметры min/max_df), не забудьте убрать l2 регуляризацию, которая по умолчанию включена
3. Если в вашем датасете целевая переменная непрерывная (например, среднее число просмотров в день), то воспользуйтесь линейной регрессией, если дискретная (положительный/отрицательный отзыв), то логистической
4. Постройте регрессию с настройкой параметра регуляризации, оцените качество при помощи соответствующих задаче метрик
5. Визуализируйте получившиеся коэффициенты регрессии (возьмите топ-50 слов). Проинтерпретируйте результаты.